In [59]:
import torch
import torch.nn.functional as F

In [105]:
x = torch.FloatTensor([[[1,2,3], [1,2,3]], [[2,3,4], [3,4,5]]])
codebook_size = 2

In [106]:
vectors = x.flatten(end_dim=1)
unique_vectors = vectors.unique(dim=0)

random_idx = torch.randperm(vectors.size(dim=0))[:codebook_size]
kmeans_centroids = vectors[random_idx]

In [107]:
torch.sum(vectors**2, dim=0) + torch.sum(kmeans_centroids**2, dim=0)

tensor([17., 41., 77.])

In [108]:
torch.matmul(vectors, kmeans_centroids.t())

tensor([[14., 14.],
        [14., 14.],
        [20., 20.],
        [26., 26.]])

In [109]:
distances = torch.sum(vectors**2, dim=1, keepdim=True) + torch.sum(kmeans_centroids**2, dim=1) - 2.0 * torch.matmul(vectors, kmeans_centroids.t())

In [110]:
dists, indexes = torch.min(distances, dim=1)

In [111]:
vectors.shape

torch.Size([4, 3])

In [113]:
for i in range(codebook_size):
    cluster_indices = torch.nonzero(indexes == i).squeeze(1)
    if cluster_indices.shape[0] > 0:
        one_hot_eq = F.one_hot(cluster_indices, num_classes=vectors.shape[0]).float()
        cluster_vals = torch.matmul(one_hot_eq, vectors)
        average = cluster_vals.sum(dim=0) / cluster_indices.shape[0]
        
        kmeans_centroids[i] = average
    else:
        kmeans_centroids[i] = # random generated
    

tensor([1.7500, 2.7500, 3.7500])
tensor([nan, nan, nan])


In [114]:
kmeans_centroids

tensor([[1.7500, 2.7500, 3.7500],
        [   nan,    nan,    nan]])

In [80]:
torch.arange(0, 5) % 3

tensor([0, 1, 2, 0, 1])